En esta parte se utilizará otro conjunto muy conocido de datos para aprendizaje, que consiste también de textos en inglés. Se trata de un conjunto de noticias de prensa de la Agencia Reuters, *Reuters-21578 *, https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

Las noticias han sido categorizadas a mano, con etiquetas de varios tipos, incluyendo *temas, lugares, organizaciones, personas* y otros criterios. Están en formato SGML, las etiquetas aparecen embebidas en el texto. Existe documentación en el archivo README de la distribución.

Nos interesaremos en el tipo de etiqueta *temas (en inglés en los textos topics)*. El objetivo es nuevamente aprender los temas a partir de los textos de las noticias, con la diferencia de que este es un problema multi-etiqueta : cada noticia puede tener varios temas. En vez de resolver el problema multi-etiqueta inicial (con más de 100 tópicos distintos) les pedimos que lo transformen según las siguientes simplificaciones: considere solo los 3 temas más frecuentes, y transforme el problema multi-etiqueta en 3 problemas de clasificación binaria

No se realiza para esta parte una especificación detallada, sino que les pedimos a Uds. que armen los pasos de una solución y definan el detalle del notebook para esta parte. Mínimamente se espera que procesen la entrada SGML, y encuentren algún modo de enfocar el problema multi-etiqueta en la versión simplificada. Se debe proponer algún modo de tratar el texto, con eventuales mejoras respecto a la vectorización de la parte 1. Se deben aplicar clasificadores y medir su performance, mínimamente con precision, recall y medida-F. Finalmente, se espera una discusión detallada de todo lo realizado y eventuales propuestas de mejora.

Puntos a tener en cuenta del conjunto de datos:
* En Topics dentro de la etiqueta <REUTERS> puede haber:
    * YES: Tiene al menos una categoría(puede existir algún error y no tener ningúna categoria)
    * NO: No tiene categoría(puede existir algún error y tener alguna categoria, supuestamente en esta versión no sucede)
    * BYPASS: No está indexado, no se verificó la categoría
* Hay etiquetas que dividen el conjunto en train y test
* Etiquetas:
    * <TOPICS>, </TOPICS> [ONCE, SAMELINE]: Lista de categorias. Si hay alguna estan delimitadas por <D></D>.
    * <UNKNOWN>, </UNKNOWN> metadatos desconocidos
    * <TEXT>, </TEXT> cuerpo del texto

In [2]:
import glob
#Cargo los archivos como lista de strings en documentos
lista_nombres_archivo = glob.glob('reuters21578/*.sgm')
documentos = []
for nombre_archivo in lista_nombres_archivo:
    with open (nombre_archivo, "r") as archivo:
        documento = archivo.read()
        documentos.append(documento)

In [3]:
print documentos[0]

<!DOCTYPE lewis SYSTEM "lewis.dtd">
<REUTERS TOPICS="YES" LEWISSPLIT="TRAIN" CGISPLIT="TRAINING-SET" OLDID="10914" NEWID="6001">
<DATE>17-MAR-1987 10:59:37.95</DATE>
<TOPICS></TOPICS>
<PLACES><D>usa</D></PLACES>
<PEOPLE></PEOPLE>
<ORGS></ORGS>
<EXCHANGES></EXCHANGES>
<COMPANIES></COMPANIES>
<UNKNOWN> 
&#5;&#5;&#5;A RM Y
&#22;&#22;&#1;f5264&#31;reute
r f BC-VALERO-ENERGY-&lt;VLO>-D   03-17 0110</UNKNOWN>
<TEXT>&#2;
<TITLE>VALERO ENERGY &lt;VLO> DEBT UPGRADED BY MOODY'S</TITLE>
<DATELINE>    NEW YORK, March 17 - </DATELINE><BODY>Moody's Investors Service Inc said it
upgraded Valero Energy Corp's 120 mln dlrs of debt.
    Moody's cited an improved leverage position that will
result from Valero's sale of its Valero Natural Gas subsidiary
to Valero Natural Gas Partners L.P., the partial sale of units
of that partnership to the public, and the simultaneous sale of
first mortgage bonds to institutional investors.
    Raised were Valero-backed tax-exempt economic development
revenue bonds of V

In [4]:
from BeautifulSoup import BeautifulSoup

raw_reuter_news = []
for documento in documentos:
    #coloco el contenido de los tags 'reuters' en otra lista: raw_reuter_news
    #esa lista contendra las noticias separadas en formato raw
    soup = BeautifulSoup(documento)
    for raw_reuter in soup('reuters'):
        raw_reuter_news.append(raw_reuter)

In [5]:
#verifico cantidad
print len(raw_reuter_news)

21578
<reuters topics="YES" lewissplit="TRAIN" cgisplit="TRAINING-SET" oldid="10914" newid="6001">
<date>17-MAR-1987 10:59:37.95</date>
<topics></topics>
<places><d>usa</d></places>
<people></people>
<orgs></orgs>
<exchanges></exchanges>
<companies></companies>
<unknown> 
&#5;&#5;&#5;A RM Y
&#22;&#22;&#1;f5264&#31;reute
r f BC-VALERO-ENERGY-&lt;VLO&gt;-D   03-17 0110</unknown>
<text>&#2;
<title>VALERO ENERGY &lt;VLO&gt; DEBT UPGRADED BY MOODY'S</title>
<dateline>    NEW YORK, March 17 - </dateline><body>Moody's Investors Service Inc said it
upgraded Valero Energy Corp's 120 mln dlrs of debt.
    Moody's cited an improved leverage position that will
result from Valero's sale of its Valero Natural Gas subsidiary
to Valero Natural Gas Partners L.P., the partial sale of units
of that partnership to the public, and the simultaneous sale of
first mortgage bonds to institutional investors.
    Raised were Valero-backed tax-exempt economic development
revenue bonds of Vincennes, Ind, to Baa-3 

In [6]:
raw_news_x_topics = {}
for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    for topics_soup in raw_reuter_soup.topics:
        #para cada topic en la lista de topics 
        #agrego la noticia a un diccionario clave topic y valor lista de noticias con ese topic.
        for topic_soup in topics_soup:
            try:
                #ya existe la lista.
                raw_news_x_topics[topic_soup].append(raw_reuter_soup)
            except KeyError:
                #es el primer valor
                raw_news_x_topics[topic_soup] = [raw_reuter_soup]

In [7]:
#ordeno el diccionario por cantidad de noticias en cada topic.
sorted_topics = sorted(raw_news_x_topics, key=lambda topic: len(raw_news_x_topics[topic]), reverse=True)
#los tres topics mas usados
for topic in sorted_topics[:3]:
        print topic,len(raw_news_x_topics[topic])

earn 3987
acq 2448
money-fx 801
